# Word2Vec實作
- 字詞所代表的意義非常多元，在不同狀況下，會代表不同意思。要把多元意思用單一向量表示，則必須要進行word embedding的動作，也就是把高維向量降為低維向量的過程
- 之前介紹過，利用分散式表示法來表達字詞向量，例如PMI、SVD..統計法..等
- 2013年神經網路盛行後，Tomas Mikolov利用神經網路訓練方式，來獲得字詞的表達向量，獲得很棒的成果。一般認為是利用神經網路模擬人類的理解能力，獲得不錯的分布空間所得到的成果。
- 本範例以維基百科wiki部分資料作範例
- 資料來源：https://dumps.wikimedia.org/zhwiki/20231201/zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2
- 利用結巴分詞(jieba)進行斷詞，gensim套件進行word2vec計算
- 本範例約需1小時長時間執行


In [3]:
!wget https://dumps.wikimedia.org/zhwiki/20231201/zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2

--2024-01-04 20:46:12--  https://dumps.wikimedia.org/zhwiki/20231201/zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2
Resolving proxy.server (proxy.server)... 10.0.0.203
Connecting to proxy.server (proxy.server)|10.0.0.203|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 230609687 (220M) [application/octet-stream]
Saving to: ‘zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2’

zhwiki-20231201-pag 100%[===================>] 219.93M  3.98MB/s    in 56s     

2024-01-04 20:47:08 (3.91 MB/s) - ‘zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2’ saved [230609687/230609687]



### opencc是繁簡轉換工具

In [5]:
# 使用 pip 安裝 opencc-python-reimplemented 套件
!pip install opencc-python-reimplemented

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels


### gensim是訓練word2vec的函式庫

In [10]:
# 建立 WikiCorpus 物件，讀取中文維基百科檔案（請確保已經下載並解壓縮）
from gensim.corpora import WikiCorpus

# 建立 WikiCorpus 物件，讀取中文維基百科檔案（請確保已經下載並解壓縮）
wiki_corpus = WikiCorpus('zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2', dictionary={})

In [12]:
wiki_corpus

In [14]:
# 取得 WikiCorpus 的文字內容，使用 iter 取得迭代器，再用 next 取得下一個元素
# 並使用切片 [:10] 取得該元素的前十個元素
next(iter(wiki_corpus.get_texts()))[:10]

['歐幾里得',
 '西元前三世紀的古希臘數學家',
 '現在被認為是幾何之父',
 '此畫為拉斐爾的作品',
 '雅典學院',
 '数学',
 '是研究數量',
 '屬於形式科學的一種',
 '數學利用抽象化和邏輯推理',
 '從計數']

## 把wiki的資料檔案，轉換成連續文字的txt檔案

In [ ]:
# 初始化計數器 text_num，用來計算處理的文章數量
text_num = 0

# 開啟檔案 'wiki_text.txt' 以進行寫入，使用 utf-8 編碼
with open('wiki_text.txt', 'w', encoding='utf-8') as f:
    # 迭代 WikiCorpus 中的每一篇文章
    for text in wiki_corpus.get_texts():
        # 將文章中的字詞用空格連接，寫入文件，並換行
        f.write(' '.join(text)+'\n')
        # 計數
        text_num += 1
        # 每處理 10000 筆資料，顯示進度訊息
        if text_num % 10000 == 0:
            print('{} articles processed.'.format(text_num))

    print('{} articles processed.'.format(text_num))

In [ ]:
# 引入 jieba 和 opencc 套件
!pip install jieba
import jieba
from opencc import OpenCC


# 初始化 OpenCC，將簡體中文轉換為繁體中文
cc = OpenCC('s2t')
# 讀取 'wiki_text.txt' 文本檔案的內容
train_data = open('wiki_text.txt', 'r', encoding='utf-8').read()
# 使用 OpenCC 將簡體中文轉換為繁體中文
train_data = cc.convert(train_data)
# 使用 jieba 套件進行中文分詞
train_data = jieba.lcut(train_data)
# 去除空白字詞
train_data = [word for word in train_data if word != '']
# 將分詞後的字詞用空格連接成字串
train_data = ' '.join(train_data)
# 將處理後的結果寫入 'seg.txt' 文本檔案
open('seg.txt', 'w', encoding='utf-8').write(train_data)

In [ ]:
# 引入 gensim 中的 word2vec
from gensim.models import word2vec


# 設定 Word2Vec 的相關參數
seed = 666
sg = 0
window_size = 10
#vector_size = 100
min_count = 1
workers = 8
#epochs = 5
batch_words = 10000

# 讀取斷詞後的資料
train_data = word2vec.LineSentence('seg.txt')
# 使用 Word2Vec 訓練模型
model = word2vec.Word2Vec(
    train_data,
    min_count=min_count,
    #size=vector_size,
    workers=workers,
    #iter=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words
)

# 儲存訓練好的 Word2Vec 模型
model.save('word2vec.model')

In [ ]:
# 引入 gensim 中的 word2vec
from gensim.models import word2vec

# 指定詞彙
string = '電腦'
# 載入訓練好的 Word2Vec 模型
model = word2vec.Word2Vec.load('word2vec.model')
# 列印指定詞彙
print(string)

# 顯示與指定詞彙最相似的詞彙及其相似度
for item in model.wv.most_similar(string):
    print(item)